In [42]:
import numpy as np
import pandas as pd

In [43]:
movies = pd.read_csv("movies.csv")

In [44]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [45]:
links = pd.read_csv("links.csv")

In [46]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [47]:
ratings = pd.read_csv("ratings.csv")

In [48]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [49]:
tags = pd.read_csv("tags.csv")

In [50]:
tags.shape

(3683, 4)

In [51]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [52]:
## data preprocessing

In [53]:
movies.isnull().sum()
movies.duplicated().sum()

0

In [54]:
links.isnull().sum()
links.duplicated().sum()

0

In [55]:
ratings.isnull().sum()
ratings.duplicated().sum()

0

In [56]:
tags.isnull().sum()
tags.duplicated().sum()

0

In [57]:
print(movies.shape)
print(tags.shape)
print(links.shape)
print(ratings.shape)

(9742, 3)
(3683, 4)
(9742, 3)
(100836, 4)


In [58]:
## creating a new dataset

In [59]:
# 1. movieId
# 2. title
# 3. genres
# 4. tag
# 5. rating
# 6. imdbId
# 7. tmdbId

In [60]:
tags_new = tags[['tag', 'movieId']]
ratings_new = ratings[['movieId', 'rating']]
tags_new

,tag,movieId
0,funny,60756
1,Highly quotable,60756
2,will ferrell,60756
3,Boxing story,89774
4,MMA,89774
...,...,...
3678,for katie,7382
3679,austere,7936
3680,gun fu,3265
3681,heroic bloodshed,3265


In [61]:
new_df = pd.merge(movies, links , on = 'movieId')

In [62]:
# different users give different ratings for same movieId. assigning average rating to to each movieId
average_ratings = ratings_new.groupby('movieId')['rating'].mean().reset_index()

# Merge the movies DataFrame with the average_ratings DataFrame based on movieId
new_df = pd.merge(new_df, average_ratings, on='movieId', how='left')

In [63]:
# concatenating tags for same movie by different users
tags_concatenated = tags_new.groupby('movieId')['tag'].apply(lambda x: ','.join(x)).reset_index()
new_df = pd.merge(new_df, tags_concatenated, on='movieId', how='left')


In [64]:
new_df.head()

,movieId,title,genres,imdbId,tmdbId,rating,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,3.920930,"pixar,pixar,fun"
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0,3.431818,"fantasy,magic board game,Robin Williams,game"
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0,3.259615,"moldy,old"
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,31357.0,2.357143,NaN
4,5,Father of the Bride Part II (1995),Comedy,113041,11862.0,3.071429,"pregnancy,remake"


In [65]:
## creating a new column that will have the preprocessed tags

In [66]:
import nltk
nltk.download("punkt")
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [67]:
new_df["new_genres"] = new_df["genres"].apply(lambda x : " ".join(x.split("|")))

In [68]:
import string
def word_tokenize(text):

    tokens = []
    start = None

    for i, char in enumerate(text):
        if char.isalnum():
            if start is None:
                start = i
        else:
            if start is not None:
                tokens.append(text[start:i])
                start = None

            tokens.append(char)

    if start is not None:
        tokens.append(text[start:])

    tks = []
    for token in tokens:
      if token != ' ':
        tks.append(token)

    return tks

In [69]:
import string

def remove_punctuation(text):
  # tokenizing the text into words
  words = word_tokenize(text)

  # defining a translation table to remove punctuation
  table = str.maketrans('', '', string.punctuation)

  # removing punctuation from each word
  stripped_words = [word.translate(table) for word in words]

  # joining the words back into a string
  clean_text = ' '.join(stripped_words)
  return clean_text

In [70]:
new_df["new_title"] = new_df["title"].apply(remove_punctuation)

In [71]:
new_df["text"] = new_df["new_title"] + new_df["new_genres"]

In [72]:
new_df.head()

,movieId,title,genres,imdbId,tmdbId,rating,tag,new_genres,new_title,text
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,3.920930,"pixar,pixar,fun",Adventure Animation Children Comedy Fantasy,Toy Story 1995,Toy Story 1995 Adventure Animation Children C...
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0,3.431818,"fantasy,magic board game,Robin Williams,game",Adventure Children Fantasy,Jumanji 1995,Jumanji 1995 Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0,3.259615,"moldy,old",Comedy Romance,Grumpier Old Men 1995,Grumpier Old Men 1995 Comedy Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,31357.0,2.357143,NaN,Comedy Drama Romance,Waiting to Exhale 1995,Waiting to Exhale 1995 Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy,113041,11862.0,3.071429,"pregnancy,remake",Comedy,Father of the Bride Part II 1995,Father of the Bride Part II 1995 Comedy


In [73]:
from nltk.corpus import stopwords
nltk.download("stopwords")
import string
# print(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [74]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [75]:
## function for data preprocessing of the comment_text column
def preprocessing(text):
  text = text.lower()
  text = nltk.word_tokenize(text)
  y = []
  for i in text:
    if i.isalnum():
      y.append(i)
  text = list(y)
  y.clear()

  for i in text:
    if i not in stopwords.words("english"):
      y.append(i)

  text = list(y)
  y.clear()

  for i in text:
    if i not in string.punctuation:
      y.append(i)

  text = list(y)
  y.clear()

  for i in text:
    y.append(ps.stem(i))


  return " ".join(y)


In [76]:
new_df["transformed_text"] = new_df["text"].apply(preprocessing)
new_df.to_csv("new_df.csv",index = False)

In [77]:
new_df.head() ## final dataset

,movieId,title,genres,imdbId,tmdbId,rating,tag,new_genres,new_title,text,transformed_text
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,3.920930,"pixar,pixar,fun",Adventure Animation Children Comedy Fantasy,Toy Story 1995,Toy Story 1995 Adventure Animation Children C...,toy stori 1995 adventur anim children comedi f...
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0,3.431818,"fantasy,magic board game,Robin Williams,game",Adventure Children Fantasy,Jumanji 1995,Jumanji 1995 Adventure Children Fantasy,jumanji 1995 adventur children fantasi
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0,3.259615,"moldy,old",Comedy Romance,Grumpier Old Men 1995,Grumpier Old Men 1995 Comedy Romance,grumpier old men 1995 comedi romanc
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,31357.0,2.357143,NaN,Comedy Drama Romance,Waiting to Exhale 1995,Waiting to Exhale 1995 Comedy Drama Romance,wait exhal 1995 comedi drama romanc
4,5,Father of the Bride Part II (1995),Comedy,113041,11862.0,3.071429,"pregnancy,remake",Comedy,Father of the Bride Part II 1995,Father of the Bride Part II 1995 Comedy,father bride part ii 1995 comedi
